In [1]:
from pyspark.sql import SparkSession
ss = SparkSession.builder.master('local').appName('pipeline').getOrCreate()

## LogisticRegression 파이프라인 구현하기
+ 예측 문제: Spark에 대한 텍스트 인지 아닌지를 분류
+ Pipeline: 머신러닝 과정에 대한 파이프라인 정의
+ HashingTF: 컬럼을 용어빈도(term-frequency) 컬럼으로 변환하는 Transformer
+ Tokenizer: 컬럼의 텍스트를 여러 개의 단어로 분할하는 Transformer

In [2]:
from pyspark.ml import Pipeline
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer

In [4]:
# 훈련 데이터에 대한 DataFrame 생성
train_df = ss.createDataFrame([
    (0, "a b c d e spark", 1.0),
    (1, "b d", 0.0),
    (2, "spark f g h", 1.0),
    (3, "hadoop mapreduce", 0.0)
], ["id", "text", "label"])

# 세 번째 column은 레이블(text가 ㅇㅇ면 1 ㅁㅁ면 0 이런식)

In [5]:
"""
    머신러닝 파이프라인 과정: Tokenizer -> HashingTF -> LogisticRegression
"""

# Tokenizer 객체 생성
tokenizer = Tokenizer(inputCol="text", outputCol="words") # 단어단위로 분류

In [6]:
# HashingTF 객체 생성
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")

In [8]:
"""
    [+] LogisticRegression 객체 생성
    - maxIter = 10
    - regParam = 0.001
"""
lr = LogisticRegression(maxIter=10, regParam=0.001)

In [9]:
# [+] Pipeline 객체 생성
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr]) 
# stages는 여러 개 단계로 구성돼있으니까 stages로 매개변수넣어줘야함)
# 순서에 맞춰 list형태로 넣어줌. tokenizer -> hashing -> lr

In [11]:
# [+] Pipeline 실행 -> 모델 생성
model = pipeline.fit(train_df)
# pipeline에 들어있는 step들 일괄 실행하는 것

In [12]:
# 테스트 데이터에 대한 DataFrame 생성
test_df = ss.createDataFrame([
    (4, "spark i j k"),
    (5, "l m n"),
    (6, "spark hadoop spark"),
    (7, "apache hadoop")
], ["id", "text"])

In [13]:
# [+] 테스트 데이터에 대한 예측
predictions = model.transform(test_df)

In [16]:
predictions.show()
# feature에 나와있는 숫자들은 feature에 해당하는 ID만들어준 것. 이것이 hasingTF의 역할

+---+------------------+--------------------+--------------------+--------------------+--------------------+----------+
| id|              text|               words|            features|       rawPrediction|         probability|prediction|
+---+------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  4|       spark i j k|    [spark, i, j, k]|(262144,[19036,68...|[0.52882855227967...|[0.62920984896684...|       0.0|
|  5|             l m n|           [l, m, n]|(262144,[1303,526...|[4.16914139534004...|[0.98477000676230...|       0.0|
|  6|spark hadoop spark|[spark, hadoop, s...|(262144,[173558,1...|[-1.8649814141189...|[0.13412348342566...|       1.0|
|  7|     apache hadoop|    [apache, hadoop]|(262144,[68303,19...|[5.41564427200183...|[0.99557321143985...|       0.0|
+---+------------------+--------------------+--------------------+--------------------+--------------------+----------+



In [ ]:
# pipeline써서 머신러닝 과정 일괄 실행.
# pipeline쓰면 중간 step의 변화 있을 경우 그 스텝(ex.tokenizer 등등)을 바꾸면 돼서 쉬움.->유지보수 편함